In [4]:
from random import random

num_complex = 4
delta = 0.2
alpha = 0.02

In [43]:
def make_grid(n):
    return [[make_simple_unit() for i in range(n)] for j in range(n)]

def make_simple_unit():
    return {'v': initialize(),'h':initialize(),'da':initialize(),'db':initialize()}

#firing is x, one weight for each connection to each complex cell
def initialize():
    return {'firing':0, 'weights':[random() for i in range(num_complex)]}

#firing is y, trace is ybar
def make_complex_unit():
    return {'firing':0,'trace':0}

def make_complex_layer():
    return [make_complex_unit for i in range(num_complex)]

def find_weight_change(trace,firing,weight):
    return alpha*trace*(firing - weight)

def find_new_trace(firing, trace):
    return (1-delta)*trace + delta*firing    

#determines which complex cell should fire
def which_firing(grid, complex_layer):
    sums = {i:0 for i in range(num_complex)}
    for i,row in grid.iteritems():
        for j,unit in row.iteritems():
            for orientation in unit:
                firing = unit[orientation]['firing']
                if (firing == 1):
                    for complex_cell_num in sums:
                        sums[complex_cell_num] += firing * unit[orientation]['weights'][complex_cell_num]
                weights = unit[orientation]['weights']
                for complex_cell_num, weight in weights.iteritems():
                    grid[i][j][orientation]['weights'][complex_cell_num] += find_weight_change(complex_layer[complex_cell_num]['trace'],firing,weight)
    inverse = [(value, key) for key, value in sums.iteritems()]
    return max(inverse)[1]

#find instances of firing feature detectors in a frame (firing feature detectors -> "ffds")
def read_frame(frame):
    ffds = []
    for row_num,row in frame.iteritems():
        for col_num,pixel in row.iteritems():
            for feature_detector,is_firing in pixel.iteritems():
                if (is_firing == 1):
                    ffds.append((row_num,col_num,feature_detector))
    return ones

def main():
    #frames = import frames from Kevin
    grid = make_grid(len(frames[0]))
    complex_layer = make_complex_layer()
    fd2orient = {0:'v', 1:'h', 2:'da', 3:'db'}
    for frame in frames:
        ffds = read_frame(frame)
        for index in ffds:
            row = index[0]
            col = index[1]
            orientation = fd2orient[index[2]]
            grid[row][col][orientation]['firing'] = 1
        complex_layer[which_firing(grid, complex_layer)]['firing'] = 1
        for complex_unit in complex_layer:
            complex_unit['trace'] = find_new_trace(complex_unit['firing'],complex_unit['trace'])    